In [ ]:
import CSV, DataFrames, Plots

base_dir = "./input/train/"
tr1_dir = "defog/"
tr1_fr  = 100
tr2_dir = "tdcsfog/"
tr2_fr  = 128

# simulating the dataLoader
all_files = readdir(base_dir*tr1_dir)

rr = rand(1:length(all_files))
@show rr
# reading in a specific file
df1 = CSV.read(base_dir*tr1_dir*all_files[rr] , DataFrames.DataFrame)
df1.Time ./= tr1_fr;

In [ ]:
events = CSV.read("./input/events.csv", DataFrames.DataFrame)

In [ ]:
nii = size(df1)[1]
sii = rand(1:nii)
aii = sii .+ (1:min(nii-sii,500*tr1_fr))

p = Plots.plot(size=(1300,400)) # layout=(3,1),
color = 2 .* df1.StartHesitation .+ df1.Turn
Plots.scatter!(p, df1.Time[aii], df1.AccV[aii], color=color[aii], label=false,title="AccV",titlefontsize=9,msw=0,ms=1)
Plots.scatter!(p, df1.Time[aii], df1.AccML[aii],color=color[aii], label=false,title="AccML",titlefontsize=9,msw=0,ms=1)
Plots.scatter!(p, df1.Time[aii], df1.AccAP[aii],color=color[aii], label=false,title="AccAP",titlefontsize=9,msw=0,ms=1)

p

In [ ]:
Plots.savefig("need_to_average.pdf")

# Transforming from Freq1 to Freq2

This section is about the transformation of a signal at frequency $\omega_1$ to a signal with frequency $\omega_2$, where frequency means the times a signal is sampled __in a second__.

In [5]:
function transform(d1, fr1::Integer, fr2::Integer)
    
    nd1 = length(d1)
    nd2 = Int(floor(nd1*fr2/fr1))
    λ   = (0:(nd2-1))*fr1/fr2
    ii1 = Int.(floor.(λ))
    λ  -= ii1
    d1  = vcat(deepcopy(d1), d1[end])

    return (1 .- λ) .* d1[1 .+ ii1] + λ .* d1[2 .+ ii1]
end


transform (generic function with 1 method)

In [ ]:

# debug
#t0 = collect(0:1.25:10)
#t,λ = transform(t0,8,10)
#@show t

accV1 = df1.AccV[aii]
time1 = df1.Time[aii]
accV2 = transform(accV1, tr1_fr, tr2_fr)
time2 = transform(time1, tr1_fr, tr2_fr)

p = Plots.scatter(time1, accV1) # df1.AccV[aii])
Plots.scatter!(p, time2, accV2 ,msw=0,ms=2) # 


In [ ]:
Plots.plot!(p,xlim=(1100,1101))

# For the second data-set

In [ ]:
# simulating the dataLoader
all_f2 = readdir(base_dir*tr2_dir)

rr2 = rand(1:length(all_f2))
@show rr2
# reading in a specific file
df2 = CSV.read(base_dir*tr2_dir*all_f2[rr2] , DataFrames.DataFrame)
df2.Time ./= tr2_fr;

In [ ]:
nii = size(df2)[1]
sii = rand(1:nii)
aii = sii .+ (1:min(nii-sii,10*tr2_fr))

p = Plots.plot(layout=(3,1),size=(800,400))
color = 2 .* df2.StartHesitation .+ df2.Turn
Plots.plot!(p[1], df2.Time[aii], df2.AccV[aii], color=color[aii], label=false,title="AccV",titlefontsize=9)
Plots.plot!(p[1], df1.Time[aii], df1.AccV[aii], label=false,title="AccV",titlefontsize=9)

Plots.plot!(p[2], df2.Time[aii], df2.AccML[aii],color=color[aii], label=false,title="AccML",titlefontsize=9)
Plots.plot!(p[2], df1.Time[aii], df1.AccML[aii], label=false,title="AccML",titlefontsize=9)

Plots.plot!(p[3], df2.Time[aii], df2.AccAP[aii],color=color[aii], label=false,title="AccAP",titlefontsize=9)
Plots.plot!(p[3], df1.Time[aii], df1.AccAP[aii], label=false,title="AccAP",titlefontsize=9)


# Your task is to:

1. build a data loader that samples the __events__ (from the "events.csv" file), copies chunks using a number $D$ of _continuous_ samples. The output is assigned as a weights into which classes are present in the series of length $D$.

1. Build a neural network that is capable to read in $D$ values and produce a $K$ outputs, where $K$ is the number of different classifications possible.

1. Evaluate the system for different values of $D$ and different architectures.


In [1]:
# maybe we don't need a struct, 
# ehh 
struct myDataLoader
    sampler
    D
    getNextBatch
end

In [2]:
import CSV, DataFrames, Plots
events = CSV.read("./input/events.csv", DataFrames.DataFrame)


function transform(d1, fr1::Integer, fr2::Integer)
    
    nd1 = length(d1)
    nd2 = Int(floor(nd1*fr2/fr1))
    λ   = (0:(nd2-1))*fr1/fr2
    ii1 = Int.(floor.(λ))
    λ  -= ii1
    d1  = vcat(deepcopy(d1), d1[end])

    return (1 .- λ) .* d1[1 .+ ii1] + λ .* d1[2 .+ ii1]
end


transform (generic function with 1 method)

In [3]:
function loadFile(path)
    CSV.read(path, DataFrames.DataFrame)
end
#load in rows in time interval [first,last]
function loadRecording(file_name, first_index, last_index)
    data = []
    path_tdcs = "./input/train/tdcsfog/"
    path_de = "./input/train/defog/"
    path_notype = "./input/train/notype/"
    frequency = 128
    try 
        data = loadFile(path_tdcs * file_name*".csv")
    catch
        try
            frequency = 100
            data = loadFile(path_de * file_name*".csv")
        catch 
            # it means we have notype
            # do we want to load them in , or what ? because the structure is a bit different, there are no event type annotations.. 
            # data = loadFile(path_notype * file_name*".csv")
        end
    end

    index_start = trunc(Int64,frequency *first_index ) + 2
    index_end = min(trunc(Int64,frequency *last_index ) + 1,size(data)[1])


    (Matrix(data[index_start:index_end,:]),frequency)

end

loadRecording (generic function with 1 method)

In [12]:
#load D samples from events.csv 
function loadSamplesFromEvents(starting_from::Int64, num_d::Int64, event_df::DataFrames.DataFrame )
    num_events = size(event_df)[1]
    ending_with = min(starting_from+num_d-1,num_events) # to avoid "index not found" exception 
    # select events to be loaded 
    # we could also do also just load randomly , if that is desired 
    chosen_events = [event_df[row,:] for row in starting_from:ending_with]
    train_data = []    
    for event in chosen_events
        series_event = loadRecording(event.Id,event.Init,event.Completion)
        push!(train_data,series_event)
        
    end
    
    train_data
end
raw_data = loadSamplesFromEvents(1000,100,events)

100-element Vector{Any}:
 ([202078.0 -0.981988130289044 … 1.0 1.0; 202079.0 -0.97797347079312 … 1.0 1.0; … ; 202484.0 -0.829361403208055 … 1.0 1.0; 202485.0 -0.83144705043725 … 1.0 1.0], 100)
 ([202488.0 -0.930769273509078 … 1.0 1.0; 202489.0 -0.942209316316154 … 1.0 1.0; … ; 204915.0 -0.97177153695919 … 1.0 1.0; 204916.0 -0.973209997167569 … 1.0 1.0], 100)
 ([204918.0 -0.962848591751373 … 1.0 1.0; 204919.0 -0.939876868811466 … 1.0 1.0; … ; 205902.0 -0.9557271474778 … 1.0 1.0; 205903.0 -0.953325871079625 … 1.0 1.0], 100)
 ([214643.0 -0.960031681283642 … 1.0 1.0; 214644.0 -0.960484574992146 … 1.0 1.0; … ; 215251.0 -1.00949827004685 … 1.0 1.0; 215252.0 -1.00779645726219 … 1.0 1.0], 100)
 ([216706.0 -0.911401281962891 … 1.0 1.0; 216707.0 -0.918129083885005 … 1.0 1.0; … ; 217348.0 -0.985466399181967 … 1.0 1.0; 217349.0 -0.986332591301684 … 1.0 1.0], 100)
 ([221034.0 -0.930542979535363 … 1.0 1.0; 221035.0 -0.908998857297648 … 1.0 1.0; … ; 221498.0 -0.943057812737738 … 1.0 1.0; 221499.0 -0.9

In [5]:
low_freq = [x==100 for (_,x) in raw_data]
raw_data[low_freq]
# I guess data should be transformed.. sometime for training ? anyway.. 

UndefVarError: UndefVarError: `raw_data` not defined